In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import soccerdata as sd

[12/16/25 19:52:08] INFO     No custom team name replacements found. You can configure these in       ]8;id=210083;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=709114;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\ASUS\soccerdata\config\teamname_replacements.json.                           

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=396691;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=554939;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_config.py#197\197]8;;\
                             C:\Users\ASUS\soccerdata\config\league_dict.json.                                     

# Find Schedule

In [ ]:
# Create a scraper class instance for the 2020/21 Premier League
year = ['1920']
for season in year:
    fbref = sd.FBref('ENG-Premier League', season)
    
    # Fetch data
    games = fbref.read_schedule()
    options = Options()
    options.add_argument('--headless=new')

    # Make Chrome look more like a real browser
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    root_url = 'https://fbref.com'
    games['match_report'] = root_url + games['match_report']
    for url in games['match_report']:
        driver = webdriver.Chrome(options = options)
        try:
            driver.get(url)
            time.sleep(3)
            
            # Get page title to identify teams
            page_title = driver.title
            # Usually: "Southampton vs Arsenal Match Summary | May 25, 2025"
            
            if 'vs.' in page_title:
                home_team = page_title.split('vs.')[0].strip()
                away_team = page_title.split('vs.')[1].split('Match')[0].strip()
                date = ' '.join(page_title.split()[-5:-2])
            else:
                home_team = "can't find vs1"
                away_team = "can't find vs2"
            
            print(f"Home: {home_team}")
            print(f"Away: {away_team}")
            print(f"Time: {time}")
            # Find all summary tables
            tables = driver.find_elements(By.CSS_SELECTOR, "table[id*='stats_'][id*='summary']")
            
            # Process home team table (first summary table without '_a')
            home_tables = [t for t in tables if '_a' not in t.get_attribute('id')]
            if home_tables:
                home_html = home_tables[0].get_attribute('outerHTML')
                df_home = pd.read_html(home_html)[0]
                
                # Add Team column
                df_home['Team'] = home_team
                
                # print(f"\n{home_team} Players with Team Column:")
                # print(df_home[[('Unnamed: 0_level_0', 'Player'), ('Team', '')]].head())
                # df_home.to_csv(f'{home_team}_players_with_team_{date}.csv', index=False)
            
            # Process away team table (summary table with '_a')
            # away_tables = [t for t in tables if '_a' in t.get_attribute('id')]
            away_tables = home_tables
            if away_tables:
                away_html = away_tables[-1].get_attribute('outerHTML')
                df_away = pd.read_html(away_html)[0]
                
                # Add Team column
                df_away['Team'] = away_team
                
                # print(f"\n{away_team} Players with Team Column:")
                # print(df_away[[('Unnamed: 0_level_0', 'Player'), ('Team', '')]].head())
                # df_away.to_csv(f'{away_team}_players_with_team_{date}.csv', index=False)
            
            # Combine both teams
            if home_tables and away_tables:
                combined_df = pd.concat([df_home, df_away], ignore_index=True)
                # print(f"\nCombined DataFrame ({len(combined_df)} players):")
                # print(combined_df[[('Unnamed: 0_level_0', 'Player'), ('Team', '')]].head(10))
                combined_df.to_csv(f'source/used/{home_team}_{away_team}_{date}.csv', index=False)
        
        finally:
            driver.quit()

                    INFO     Saving cached data to C:\Users\ASUS\soccerdata\data\FBref               ]8;id=574162;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=404969;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_common.py#263\263]8;;\

[12/16/25 19:52:09] WARNING  c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\fbref.py: ]8;id=58329;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=634891;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             165: FutureWarning: The behavior of DataFrame concatenation with empty                
                             or all-NA entries is deprecated. In a future version, this will no                    
                             longer exclude empty or all-NA columns when determining the result                    
                             dtypes. To retain the old behavior, exclude the relevant entries                      
                             before the concat operation.                                                          
                               pd.concat(dfs)                                                                      
                                                                                                                   

Home: Liverpool
Away: Norwich City
Time: <module 'time' (built-in)>


[12/16/25 19:52:31] WARNING  C:\Users\ASUS\AppData\Local\Temp\ipykernel_34132\380731871.py:46:      ]8;id=426599;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=781022;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             FutureWarning: Passing literal html to 'read_html' is deprecated and                  
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                               df_home = pd.read_html(home_html)[0]                                                
                                                                                                                   

                    WARNING  C:\Users\ASUS\AppData\Local\Temp\ipykernel_34132\380731871.py:60:      ]8;id=261288;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=28747;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             FutureWarning: Passing literal html to 'read_html' is deprecated and                  
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                               df_away = pd.read_html(away_html)[0]                                                
                                                                                                                   

...

[12/16/25 21:02:32] WARNING  C:\Users\ASUS\AppData\Local\Temp\ipykernel_34132\380731871.py:46:      ]8;id=923408;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=17029;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             FutureWarning: Passing literal html to 'read_html' is deprecated and                  
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                               df_home = pd.read_html(home_html)[0]                                                
                                                                                                                   

                    WARNING  C:\Users\ASUS\AppData\Local\Temp\ipykernel_34132\380731871.py:60:      ]8;id=880320;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=560230;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             FutureWarning: Passing literal html to 'read_html' is deprecated and                  
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                               df_away = pd.read_html(away_html)[0]                                                
                                                                                                                   

Home: West Ham United
Away: Aston Villa
Time: <module 'time' (built-in)>


[12/16/25 21:02:43] WARNING  C:\Users\ASUS\AppData\Local\Temp\ipykernel_34132\380731871.py:46:      ]8;id=460354;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=321862;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             FutureWarning: Passing literal html to 'read_html' is deprecated and                  
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                               df_home = pd.read_html(home_html)[0]                                                
                                                                                                                   

                    WARNING  C:\Users\ASUS\AppData\Local\Temp\ipykernel_34132\380731871.py:60:      ]8;id=977260;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=904204;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             FutureWarning: Passing literal html to 'read_html' is deprecated and                  
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                               df_away = pd.read_html(away_html)[0]                                                
                                                                                                                   